In [31]:
import pickle
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import aidd.sys.config as cfg
from aidd.utils.data_io import read_data, save_data
from aidd.utils.data_io import read_pickle, save_pickle

In [33]:
class Scaling:
    def __init__(self, pdf=None):
        self.df = read_data('PP,SL', dtype={'CONS_ID': str}) if pdf is None else pdf
        self.data = {}
        self.sdata = {}
        self._run()
        
    def _run(self):
        self._split_Xy()
        for pc in cfg.DATA_PC_TYPE[1:]:
            self._split_pc(pc_mode=pc)
        for pc in cfg.DATA_PC_TYPE:
            self._split_train_test(pc_mode=pc)
        self._scaling()
        
    def _split_Xy(self):
        df = self.df
        
        # TARGET 컬럼 지정
        target_col = 'TOTAL_CONS_COST'
        # 학습 컬럼 지정
        training_cols = df.columns[5:].tolist()
        # X, y값 분리
        X = df[training_cols].copy()
        y = df[target_col].copy()
        print(f'학습대상 속성 전체 크기: {X.shape}')
        
        self.data['X_ALL'] = X
        self.data['y_ALL'] = y
        
        # 학습 데이터 저장
        save_data(X, file_code='SCALING,X,ALL')
        save_data(y, file_code='SCALING,y,ALL')
        
    def _split_pc(self, pc_mode='1'):
        _X = self.data['X_ALL']
        _y = self.data['y_ALL']
        
        if pc_mode == '1':
            conditions = _X.POLE_CNT == 1
            X, y = _X[conditions], _y[conditions]
        else:
            conditions = _X.POLE_CNT != 1
            X, y = _X[conditions], _y[conditions]
        print(f'PC_MODE[{pc_mode}] X Size: {X.shape}')
        
        self.data[f'X_{pc_mode}'] = X
        self.data[f'y_{pc_mode}'] = y
        
        # 학습 데이터 저장
        save_data(X, file_code=f'SCALING,X,{pc_mode}')
        save_data(y, file_code=f'SCALING,y,{pc_mode}')
    
    def _split_train_test(self, pc_mode='ALL'):
        X = self.data[f'X_{pc_mode}']
        y = self.data[f'y_{pc_mode}']
        
        train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
        msg = f'PC_MODE[{pc_mode}] Total{X.shape}, Train{train_X.shape}, ' \
              f'Test{test_X.shape}'
        print(msg)
        self.data[f'TRAIN_X_{pc_mode}'] = train_X
        self.data[f'TRAIN_y_{pc_mode}'] = train_y
        self.data[f'TEST_X_{pc_mode}'] = test_X
        self.data[f'TEST_y_{pc_mode}'] = test_y
        
    def _scaling(self):
        for pc in cfg.DATA_PC_TYPE:
            self._scaling_main(pc_mode=pc)
            
    def _scaling_main(self, pc_mode='ALL'):
        train_X = self.data[f'TRAIN_X_{pc_mode}']
        test_X = self.data[f'TEST_X_{pc_mode}']
        
        cols = train_X.columns.tolist()
        scaler = StandardScaler()
        train_sX = scaler.fit_transform(train_X)
        test_sX = scaler.transform(test_X)

        train_sX_df = pd.DataFrame(train_sX, columns=cols)
        test_sX_df = pd.DataFrame(test_sX, columns=cols)
        
        # 클래스에 저장
        self.sdata[f'TRAIN_X_{pc_mode}'] = train_sX_df
        self.sdata[f'TRAIN_y_{pc_mode}'] = self.data[f'TRAIN_y_{pc_mode}']
        self.sdata[f'TEST_X_{pc_mode}'] = train_sX_df
        self.sdata[f'TEST_y_{pc_mode}'] = self.data[f'TEST_y_{pc_mode}']
        
        # 이 부분은 굳이 저장할 필요가 있나 싶음
        save_data(train_sX_df, file_code=f'SCALING,TRAIN_X,{pc_mode}')
        save_data(self.data[f'TRAIN_y_{pc_mode}'], f'SCALING,TRAIN_y,{pc_mode}')
        save_data(test_sX_df, file_code=f'SCALING,TEST_X,{pc_mode}')
        save_data(self.data[f'TEST_y_{pc_mode}'], f'SCALING,TEST_y,{pc_mode}')
        
        # 스케일러 저장
        save_pickle(scaler, file_code=f'DUMP,SCALER,{pc_mode}')

In [34]:
batch_scaling = Scaling()

학습대상 속성 전체 크기: (14729, 104)
PC_MODE[1] X Size: (8750, 104)
PC_MODE[N1] X Size: (5979, 104)
PC_MODE[ALL] Total(14729, 104), Train(11783, 104), Test(2946, 104)
PC_MODE[1] Total(8750, 104), Train(7000, 104), Test(1750, 104)
PC_MODE[N1] Total(5979, 104), Train(4783, 104), Test(1196, 104)


In [35]:

def online_scaling(row=None):
    df = read_data('ONLINE') if pdf is None else pdf
    scaler = {pc: read_pickle(f'DUMP,SCALER,{pc}') for pc in cfg.DATA_PC_TYPE}
    
    target_col = 'TOTAL_CONS_COST'
    training_cols = df.columns[5:].tolist()
    
    X = df[training_cols].copy()
    y = df[target_col].copy()
    
    # 각 레코드별로 처리
    sdata = {}
    for idx, row in df.iterrows():
        if row['POLE_CNT'] == 1:
            pass
    return df

In [36]:
df = online_scaling()

In [39]:
for idx, row in df.iterrows():
    print(row['POLE_CNT'])
    print(row)
    print(type(row))

4
CONS_ID                   477420204194
TOTAL_CONS_COST                9076645
LAST_MOD_DATE      2020-12-23 11:50:39
LAST_MOD_EID                MDE1900011
OFFICE_NAME                       단양지사
                          ...         
SL_SPEC_60.0                       0.0
SL_SPEC_70.0                       0.0
SL_SPEC_100.0                      0.0
SL_SPEC_120.0                      0.0
SL_SPEC_240.0                      0.0
Name: 0, Length: 109, dtype: object
<class 'pandas.core.series.Series'>
1
CONS_ID                   475920223725
TOTAL_CONS_COST                2176378
LAST_MOD_DATE      2022-11-24 10:42:22
LAST_MOD_EID                MDP2100032
OFFICE_NAME                       음성지사
                          ...         
SL_SPEC_60.0                       0.0
SL_SPEC_70.0                       0.0
SL_SPEC_100.0                      0.0
SL_SPEC_120.0                      0.0
SL_SPEC_240.0                      0.0
Name: 1, Length: 109, dtype: object
<class 'pandas.core.series.Ser